In [2]:
# https://carla.readthedocs.io/en/0.9.12/adv_agents/#:~:text=set_destination(end_location%2C%20start_location%3DNone,use%20the%20current%20agent%20location
#BEHAVIOUR AGENT CON SETTAGGIO DESTINAZIONE

#all imports
import carla #the sim library itself
import cv2 #to work with images from cameras
import time # towork with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import random
import sys
import threading

from agents.navigation.global_route_planner import GlobalRoutePlanner
from agents.navigation.behavior_agent import BehaviorAgent


#ausiliarie per disegnare simboli sulla mappa
def draw_entire_route(route, duration):
    for waypoint in route:
        world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=True, color=carla.Color(r=0, g=0, b=255), life_time=duration)

def erase_route(route, duration):
    for waypoint in route:
        world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=True, color=carla.Color(r=255, g=255, b=255), life_time=duration)

def draw_symbol(wp, duration, symbol, r, g, b):
    world.debug.draw_string(wp.transform.location, symbol, draw_shadow=False, color=carla.Color(r, g, b), life_time=duration)


#DISEGNA DISEGNA 10 SIMBOLI DI FRONTE AL VEICOLO LUNGO IL PERCORSO
def draw_route(wp, route,seconds=3.0): 
    if len(route)-wp <25: # A 25 PUNTI DALLA FINE COLORA I WP DI ROSSO
        draw_colour = carla.Color(r=255, g=0, b=0)
    else:
        draw_colour = carla.Color(r=0, g=255, b=0) #ALTRIMENTI DI VERDE
    for i in range(10): #STAMPA 10 SIMBOLI DI FRONTE AL VEICOLO IN BASE AL COLORE PRECEDEMENTE SCELTO
        if wp+i<len(route)-2:
            world.debug.draw_string(route[wp+i][0].transform.location, '^', draw_shadow=False, color=draw_colour, life_time=seconds, persistent_lines=True)


# CONNESSIONE AL SIMULATORE
client = carla.Client('localhost', 2000)

#RECUPERA ASSETS DEL SIMULATORE
world = client.get_world()
bp_lib = world.get_blueprint_library() 

#TUTTI I PUNTI DI SPAWN (CONSIGLIATI) DELLA MAPPA
spawn_points = world.get_map().get_spawn_points()

#######################################################SEZIONE GENERAZIONE TRAFFICO ###########################################################

# GENERA NV AUTO CASUALI ABILITANDONE L'AGENT BEHAVIOURAL
NV = 20
#attiva disattiva traffico -> 1 per abilitare
TRAFFIC_ON = 1
#per abilitare scritta NPC sulle machine del traffico -> 1 per abilitare
TOGGLE_NPC_DIPLAY = 1

agent_list = []
vehicle_list = []

def draw_symbol(wp, duration, symbol, r, g, b):
    world.debug.draw_string(wp.transform.location, symbol, draw_shadow=False, color=carla.Color(r, g, b), life_time=duration)

#routine che per ogni veicolo in input avvia un agente in un thread dedicato
def roam_vehicle(in_vehicle):
    #RECUPERO I WAYPOINTS CORRISPONDENTI RISPETTIVAMENTE ALLE COORDINATE DI SPWN DI SPAWN DEL VEICOLO E DI DESTINAZIONE DELL'AGENTE
    start_location = in_vehicle.get_location()
    start_waypoint = world.get_map().get_waypoint(start_location)
    
    #PUNTO A CASO DALLA MAPPA COME DESTINAZIONE, SCEGLIE DI DEFAULT IL PERCORSO PIU' BREVE VERSO QUEL PUNTO
    dest_location = random.choice(spawn_points).location
    dest_waypoint = world.get_map().get_waypoint(dest_location)
    
    #L' AGENTE BEHAVIOUR REAGISCE A SEMAFORI E SEGNALI DI STOP, EVITA PEDONI, SEGUE LE MACCHINE 
    #3 POSSIBILI PROFILI AL MOMENTO: cautious, normal, aggressive -> https://carla.readthedocs.io/en/0.9.12/adv_agents/#behavior-types
    agent = BehaviorAgent(in_vehicle, behavior='aggressive')
    
    #RECUPERA IL PERCORSO CHE SEGUIRA' L'AGENTE IN UNA LISTA DI WAYPOINTS
    route = agent.trace_route(start_waypoint, dest_waypoint)
    
    #DESTINAZIONE AGENT
    agent.set_destination(dest_location)
    #lista agent
    agent_list.append(agent)
    
    #print(f"Avvio agente per veicolo: {in_vehicle}")
    #spectator = world.get_spectator() 
    #transform = carla.Transform(first_v.get_transform().transform(carla.Location(x=-4,z=2.5)),first_v.get_transform().rotation) 
    #spectator.set_transform(transform)

    #MAIN THREAD LOOP:
    while True:
        global stop_threads
        if agent.done() or stop_threads: #SE L'AGENT HA CONCLUSO COLORA DI BIANCO IL PERCORSO COMPLETATO
            print(f"Il veicolo {in_vehicle} ha raggiunto la sua destinazione (Stop forzato={stop_threads})")
            in_vehicle.destroy()
            break
        #APPLICA IL CONTROLO DELL'AGENTE
        in_vehicle.apply_control(agent.run_step())
        curr_wp = world.get_map().get_waypoint(in_vehicle.get_location())
        
        if TOGGLE_NPC_DIPLAY == 1:
            draw_symbol(curr_wp, 0.01, 'NPC', 255, 255, 255) 

first_v = None
npc_vehicle = None

if TRAFFIC_ON == 1:
    
    print(f"Inizio generazione di {NV} veicoli casuali")
    for i in range(NV):
        vehicle_bp = random.choice(bp_lib.filter('vehicle')) 
        npc_vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
        
        if i == 0: 
            first_v = npc_vehicle
            
        if npc_vehicle != None :
            #print(f"veicolo n {i+1}")
            #print(npc_vehicle)
            vehicle_list.append(npc_vehicle)
    
    # Creazione e avvio dei thread per ciasucn agente
    stop_threads = False
    threads = []
    for i in vehicle_list:
        # Crea un thread per ogni veicolo
        thread = threading.Thread(target=roam_vehicle, args=(i,))
        threads.append(thread)
        thread.start()
    
    print(f"Fine generazione di {NV} veicoli casuali")
else: 
    print("TRAFFICO DISABILITATO")

#######################################################FINE SEZIONE GENERAZIONE TRAFFICO######################################################

############################################### SETTAGGIO EGO VEHICLE #################################################################

# ISTANZIA L'EGO VEHICLE IN UN PUNTO A CASO DELLA MAPPA
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020') 
start_spawn = random.choice(spawn_points)
vehicle = world.try_spawn_actor(vehicle_bp, start_spawn)

#CENTRA TELECAMERA SUL VEICOLO
spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform)

#RECUPERO I WAYPOINTS CORRISPONDENTI RISPETTIVAMENTE ALLE COORDINATE DI SPWN DI SPAWN DEL VEICOLO E DI DESTINAZIONE DELL'AGENTE
start_location = vehicle.get_location()
start_waypoint = world.get_map().get_waypoint(start_location)

#PUNTO A CASO DALLA MAPPA COME DESTINAZIONE, SCEGLIE DI DEFAULT IL PERCORSO PIU' BREVE VERSO QUEL PUNTO
dest_location = random.choice(spawn_points).location
dest_waypoint = world.get_map().get_waypoint(dest_location)

#L' AGENTE BEHAVIOUR REAGISCE A SEMAFORI E SEGNALI DI STOP, EVITA PEDONI, SEGUE LE MACCHINE 
#3 POSSIBILI PROFILI AL MOMENTO: cautious, normal, aggressive -> https://carla.readthedocs.io/en/0.9.12/adv_agents/#behavior-types
agent = BehaviorAgent(vehicle, behavior='cautious')

#RECUPERA IL PERCORSO CHE SEGUIRA' L'AGENTE IN UNA LISTA DI WAYPOINTS
route = agent.trace_route(start_waypoint, dest_waypoint)

'''
draw_entire_route(route, 120)
draw_symbol(start_waypoint, 120, '<<<<<!!!!!-START-!!!!!>>>>>', 0, 255, 0)
draw_symbol(dest_waypoint, 120, '<<<<<!!!!!-END-!!!!!>>>>>', 255, 0, 0)
'''

#DESTINAZIONE AGENT
agent.set_destination(dest_location)

###############################################      INIZIALIZZAZIONE SENSORI      #####################################################
# SENSORE TELECAMERA PER FINESTRA PYGAME
camera_bp = bp_lib.find('sensor.camera.rgb') 
camera_init_trans = carla.Transform(carla.Location(z=2))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

# sensore di posizione
gnss_bp = bp_lib.find('sensor.other.gnss')
gnss_sensor = world.spawn_actor(gnss_bp, carla.Transform(), attach_to=vehicle)

# sensore di misurazione movimento inerziale 
imu_bp = bp_lib.find('sensor.other.imu')
imu_sensor = world.spawn_actor(imu_bp, carla.Transform(), attach_to=vehicle)

# sensore rilevazione collisione
collision_bp = bp_lib.find('sensor.other.collision')
collision_sensor = world.spawn_actor(collision_bp, carla.Transform(), attach_to=vehicle)

# sensore invasione di corsia
lane_inv_bp = bp_lib.find('sensor.other.lane_invasion')
lane_inv_sensor = world.spawn_actor(lane_inv_bp, carla.Transform(), attach_to=vehicle)

# attributi sensore ostacoli
obstacle_bp = bp_lib.find('sensor.other.obstacle')
obstacle_bp.set_attribute('hit_radius','0.5')
obstacle_bp.set_attribute('distance','50')
obstacle_sensor = world.spawn_actor(obstacle_bp, carla.Transform(), attach_to=vehicle)

#################################### DEFINIZIONE FUNZIONI DI CALLBACK PER CIASCUN SENSORE #########################################
def rgb_callback(image, data_dict):
    data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def gnss_callback(data, data_dict):
    data_dict['gnss'] = [data.latitude, data.longitude]

def imu_callback(data, data_dict):
    data_dict['imu'] = {
        'gyro': data.gyroscope,
        'accel': data.accelerometer,
        'compass': data.compass
    }
    
def lane_inv_callback(event, data_dict):
    data_dict['lane_invasion'] == True
    
def collision_callback(event, data_dict):
    data_dict['collision'] = True
    
#disegna un cerchio sull'ostacolo rilevato   
def obstacle_callback(event, data_dict, camera, k_mat):
    if 'static' not in event.other_actor.type_id: #salta rilevazione ostacoli statici come marciapiedi, strade, cassete posta ecc..
        data_dict['obstacle'].append({'transform': event.other_actor.type_id, 'frame': event.frame})
        
    world_2_camera = np.array(camera.get_transform().get_inverse_matrix())
    image_point = get_image_point(event.other_actor.get_transform().location, k_mat, world_2_camera)
    if  0 < image_point[0] < image_w and 0 < image_point[1] < image_h:
        cv2.circle(data_dict['rgb_image'], tuple(image_point), 10, (0,0,255), 3)

# strutture dati ausiliare per poriezione coordinate su schermo
def build_projection_matrix(w, h, fov):
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0))
    K = np.identity(3)
    K[0, 0] = K[1, 1] = focal
    K[0, 2] = w / 2.0
    K[1, 2] = h / 2.0
    return K

def get_image_point(loc, K, w2c):
        # Calculate 2D projection of 3D coordinate

        # Format the input coordinate (loc is a carla.Position object)
        point = np.array([loc.x, loc.y, loc.z, 1])
        # transform to camera coordinates
        point_camera = np.dot(w2c, point)

        # conversione coordinate UE4's verso quelle standard
        # (x, y ,z) -> (y, -z, x)
        # rimuovendo la quarta componente
        point_camera = [point_camera[1], -point_camera[2], point_camera[0]]

        # now project 3D->2D using the camera matrix
        point_img = np.dot(K, point_camera)
        # normalize
        point_img[0] /= point_img[2]
        point_img[1] /= point_img[2]

        return tuple(map(int, point_img[0:2]))
    
world_2_camera = np.array(camera.get_transform().get_inverse_matrix())

# Get the attributes from the camera
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()
fov = camera_bp.get_attribute("fov").as_float()

# Calculate the camera projection matrix to project from 3D -> 2D
K = build_projection_matrix(image_w, image_h, fov)

#################################################  AVVIO DEI SENSORI ############################################
collision_counter = 20
lane_invasion_counter = 20
sensor_data = {'rgb_image': np.zeros((image_h, image_w, 4)),
               'collision': False,
               'lane_invasion': False,
               'gnss': [0,0],
               'obstacle': [],
               'imu': {
                    'gyro': carla.Vector3D(),
                    'accel': carla.Vector3D(),
                    'compass': 0
                }}

# OpenCV window with initial data
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE) 
cv2.imshow('RGB Camera', sensor_data['rgb_image'])
cv2.waitKey(1)

# Start sensors recording data
camera.listen(lambda image: rgb_callback(image, sensor_data))
collision_sensor.listen(lambda event: collision_callback(event, sensor_data))
gnss_sensor.listen(lambda event: gnss_callback(event, sensor_data))
imu_sensor.listen(lambda event: imu_callback(event, sensor_data))
lane_inv_sensor.listen(lambda event: lane_inv_callback(event, sensor_data))
obstacle_sensor.listen(lambda event: obstacle_callback(event, sensor_data, camera, K))

# Some parameters for text on screen
font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,50)
fontScale              = 0.5
fontColor              = (255,255,255)
thickness              = 2
lineType               = 2

# Disegna la bussola (in radianti) con una linea direzionata verso i punti cardinali
def draw_compass(img, theta):
    compass_center = (700, 100)
    compass_size = 50
    
    cardinal_directions = [
        ('N', [0,-1]),
        ('E', [1,0]),
        ('S', [0,1]),
        ('W', [-1,0])
    ]
    for car_dir in cardinal_directions:
        cv2.putText(sensor_data['rgb_image'], car_dir[0], 
        (int(compass_center[0] + 1.2 * compass_size * car_dir[1][0]), int(compass_center[1] + 1.2 * compass_size * car_dir[1][1])), 
        font, 
        fontScale,
        fontColor,
        thickness,
        lineType)
    
    compass_point = (int(compass_center[0] + compass_size * math.sin(theta)), int(compass_center[1] - compass_size * math.cos(theta)))
    cv2.line(img, compass_center, compass_point, (255, 255, 255), 3)


Inizio generazione di 20 veicoli casuali
Fine generazione di 20 veicoli casuali


In [ ]:
##################################################    MAIN CODE      ###############################################################

#MAIN LOOP DI GUIDA:
while True:
    #SE L'AGENT HA CONCLUSO COLORA DI BIANCO IL PERCORSO COMPLETATO
    if agent.done() or cv2.waitKey(1) == ord('q'):
        vehicle.destroy()
        print("Destinazione raggiunta, Fine simulazione")
        break
    
    ################################### SEZIONE CATTURA DATI DAI SENSORI  ################################################
    # Latitudine dal sensore GNSS
    cv2.putText(sensor_data['rgb_image'], 'Lat: ' + str(sensor_data['gnss'][0]), 
    (10,30), 
    font, 
    fontScale,
    fontColor,
    thickness,
    lineType)
    
    # Longitudine dal sensore GNSS
    cv2.putText(sensor_data['rgb_image'], 'Long: ' + str(sensor_data['gnss'][1]), 
    (10,50), 
    font, 
    fontScale,
    fontColor,
    thickness,
    lineType)
    
    # Calcola il vettore di accelerazione sottraendo la gravità (-9.81)
    accel = sensor_data['imu']['accel'] - carla.Vector3D(x=0,y=0,z=9.81)
    
    # Stampa lintensità di accelerazione
    cv2.putText(sensor_data['rgb_image'], 'Accel: ' + str(accel.length()), 
    (10,70), 
    font, 
    fontScale,
    fontColor,
    thickness,
    lineType)
    
    # Giroscopio
    cv2.putText(sensor_data['rgb_image'], 'Gyro: ' + str(sensor_data['imu']['gyro'].length()), 
    (10,100), 
    font, 
    fontScale,
    fontColor,
    thickness,
    lineType)
    
    # bussola, nord è zero radianti
    cv2.putText(sensor_data['rgb_image'], 'Compass: ' + str(sensor_data['imu']['compass']), 
    (10,120), 
    font, 
    fontScale,
    fontColor,
    thickness,
    lineType)
    
    # Draw the compass
    draw_compass(sensor_data['rgb_image'], sensor_data['imu']['compass'])
    
    # stampa 'COLLISION' quando il flag del sensore è true, dura 20 fotogrammi
    if sensor_data['collision']:
        collision_counter -= 1
        if collision_counter <= 1:
            sensor_data['collision'] = False
        cv2.putText(sensor_data['rgb_image'], 'COLLISION', 
        (250, 300), 
        cv2.FONT_HERSHEY_SIMPLEX, 
        2,
        (255,255,255),
        3,
        2)
    else:
        collision_counter = 20
        
    # stampa 'LANE INVASION' uando il flag del sensore è true, dura 20 fotogrammi
    if sensor_data['lane_invasion']:
        lane_invasion_counter -= 1
        if lane_invasion_counter <= 1:
            sensor_data['lane_invasion'] = False
        cv2.putText(sensor_data['rgb_image'], 'LANE INVASION', 
        (190, 350), 
        cv2.FONT_HERSHEY_SIMPLEX, 
        2,
        (255,255,255),
        3,
        2)
    else:
        lane_invasion_counter = 20
     
    # mostra il POV del veicolo in una finestra separata
    cv2.imshow('RGB Camera', sensor_data['rgb_image'])
    
    # Break the loop if the user presses the Q key
    if cv2.waitKey(1) == ord('q'):
        break

    ################################  ATTIVAZIONE CONTINUA DELL'AGENTE ##########################
    vehicle.apply_control(agent.run_step())
    curr_wp = world.get_map().get_waypoint(vehicle.get_location())
    
    #ogni secondo disegna sia il percorso che i punti di start e fine
    draw_entire_route(route, 1)
    draw_symbol(start_waypoint, 1, '<<<<<!!!!!-START-!!!!!>>>>>', 0, 255, 0)
    draw_symbol(dest_waypoint, 1, '<<<<<!!!!!-END-!!!!!>>>>>', 255, 0, 0)
    draw_symbol(curr_wp, 0.01, '!!!CAR!!!', 255, 0, 0) 

##############################DISTRUGGE TUTTI GLI ASSET  (SENSORI E VEICOLI)  DELLA SIMULAZIONE #################################

#PER 10 SECONDI COLORA DI BIANCO I SIMBOLI DEL PERCORSO COMPLETATO
erase_route(route, 10)
draw_symbol(start_waypoint, 10, '<<<<<!!!!!-START-!!!!!>>>>>', 255, 255, 255)
draw_symbol(dest_waypoint, 10, '<<<<<!!!!!-END-!!!!!>>>>>', 255, 255, 255)
#cv2.destroyWindow("RGB Camera")

#ferma tutti i threads dei veicoli NPC
stop_threads = True
camera.stop()
collision_sensor.stop()
gnss_sensor.stop()
imu_sensor.stop()
lane_inv_sensor.stop()
obstacle_sensor.stop()
cv2.destroyAllWindows()
#for actor in world.get_actors().filter('*vehicle*'):
#    actor.destroy()
for actor in world.get_actors().filter('*sensor*'):
    actor.destroy()

print("SIMULAZIONE COMPLETATA")
